<a href="https://colab.research.google.com/github/Leo140102/desafioPython/blob/main/desafioPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import hashlib
import requests
import pandas as pd
import sqlite3
from google.colab import userdata
from pandas import DataFrame

#Obtém as chaves de API do usuário.
def get_api_keys():
    return userdata.get('PUBLIC_KEY'), userdata.get('PRIVATE_KEY')

#Gera o hash MD5 necessário para autenticação na API.
def generate_md5_hash(public_key, private_key):
    ts = "1"  # Timestamp fixo, conforme exigido pela API
    hash_input = f"{ts}{private_key}{public_key}"
    return hashlib.md5(hash_input.encode('utf-8')).hexdigest()

 #Faz uma requisição GET à API e retorna os dados em formato JSON.
def fetch_data_from_api(endpoint, public_key, hash_md5):
    base_url = "http://gateway.marvel.com"
    url = f"{base_url}/{endpoint}?ts=1&apikey={public_key}&hash={hash_md5}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

 #Cria as tabelas no banco de dados, caso não existam.
def create_tables(cursor):
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS characters (
            id INTEGER PRIMARY KEY,
            name TEXT,
            description TEXT
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS comics (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            comic_title TEXT
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS events (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            description TEXT,
            start DATE,
            end DATE
        )
    """)

 #Insere os dados na tabela especificada do banco de dados.
def insert_data_into_db(cursor, data, table_name):
    for item in data:
        if table_name == 'characters':
            cursor.execute("""
                INSERT OR IGNORE INTO characters (id, name, description)
                VALUES (?, ?, ?)
            """, (item["id"], item["name"], item["description"]))
        elif table_name == 'comics':
            cursor.execute("""
                INSERT INTO comics (comic_title)
                VALUES (?)
            """, (item["title"],))
        elif table_name == 'events':
            cursor.execute("""
                INSERT OR IGNORE INTO events (title, description, start, end)
                VALUES (?, ?, ?, ?)
            """, (item["title"], item["description"], item["start"], item["end"]))

#Função principal que executa todas as etapas do processo.
def main():
    try:
        public_key, private_key = get_api_keys()
        hash_md5 = generate_md5_hash(public_key, private_key)

        # Fetching data from Marvel API
        characters_data = fetch_data_from_api("v1/public/characters", public_key, hash_md5)["data"]["results"]
        comics_data = fetch_data_from_api("v1/public/comics", public_key, hash_md5)["data"]["results"]
        events_data = fetch_data_from_api("v1/public/events", public_key, hash_md5)["data"]["results"]

        # Conectando ao banco de dados SQLite
        with sqlite3.connect('marvel.db') as conn:
            cursor = conn.cursor()

            # Criando as tabelas no banco de dados
            create_tables(cursor)

            # Inserindo os dados nas tabelas
            insert_data_into_db(cursor, characters_data, 'characters')
            insert_data_into_db(cursor, comics_data, 'comics')
            insert_data_into_db(cursor, events_data, 'events')

        # Lendo as tabelas com pandas
        with sqlite3.connect('marvel.db') as conn:
            personagens_df = pd.read_sql_query("SELECT * FROM characters", conn)
            comics_df = pd.read_sql_query("SELECT * FROM comics", conn)
            events_df = pd.read_sql_query("SELECT * FROM events", conn)

        # Exibindo os resultados
        print("Tabela de Personagens:")
        print(personagens_df)

        print("\nTabela de Quadrinhos:")
        print(comics_df)

        print("\nTabela de Eventos:")
        print(events_df)

    except requests.exceptions.RequestException as e:
        print(f"Erro ao fazer a requisição à API: {e}")
    except sqlite3.DatabaseError as e:
        print(f"Erro no banco de dados: {e}")
    except Exception as e:
        print(f"Erro inesperado: {e}")


main()


Tabela de Personagens:
         id                        name  \
0   1009144                      A.I.M.   
1   1009146  Abomination (Emil Blonsky)   
2   1009148               Absorbing Man   
3   1009149                       Abyss   
4   1009150                  Agent Zero   
5   1010354                Adam Warlock   
6   1010699                 Aaron Stack   
7   1010846        Aegis (Trey Rollins)   
8   1010903   Abyss (Age of Apocalypse)   
9   1011031              Agent X (Nijo)   
10  1011136    Air-Walker (Gabriel Lan)   
11  1011175                      Aginar   
12  1011198             Agents of Atlas   
13  1011266                Adam Destine   
14  1011297                 Agent Brand   
15  1011334                     3-D Man   
16  1012717             Agatha Harkness   
17  1016823      Abomination (Ultimate)   
18  1017100                A-Bomb (HAS)   
19  1017851                 Aero (Aero)   

                                          description  
0   AIM is a terr